<a href="https://colab.research.google.com/github/ravgupta11/Machine-Learning-Deep-Learning-Matrix/blob/main/DecisionTree(Entropy_and_IG)WithoutSkilearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
  df = pd.read_csv('house_data.csv', names = ['in_sf','beds','bath','price','year_built','sqft','price_per_sqft','elevation'],
                   delimiter= ',',
                   converters={'in_sf':float,'beds':float, 'bath':float, 'price':float, 'year_built':int, 'sqft':float, 'price_per_sqft':float, 'elevation':float})

  df

,in_sf,beds,bath,price,year_built,sqft,price_per_sqft,elevation
0,0.0,2.0,1.0,999000.0,1960,1000.0,999.0,10.0
1,0.0,2.0,2.0,2750000.0,2006,1418.0,1939.0,0.0
2,0.0,2.0,2.0,1350000.0,1900,2150.0,628.0,9.0
3,0.0,1.0,1.0,629000.0,1903,500.0,1258.0,9.0
4,0.0,0.0,1.0,439000.0,1930,500.0,878.0,10.0
...,...,...,...,...,...,...,...,...
487,1.0,5.0,2.5,1800000.0,1890,3073.0,586.0,76.0
488,1.0,2.0,1.0,695000.0,1923,1045.0,665.0,106.0
489,1.0,3.0,2.0,1650000.0,1922,1483.0,1113.0,106.0
490,1.0,1.0,1.0,649000.0,1983,850.0,764.0,163.0


In [ ]:
df['Class'] = df['in_sf']
df.drop(['in_sf'], axis = 1, inplace = True)
df

,beds,bath,price,year_built,sqft,price_per_sqft,elevation,Class
0,2.0,1.0,999000.0,1960,1000.0,999.0,10.0,0.0
1,2.0,2.0,2750000.0,2006,1418.0,1939.0,0.0,0.0
2,2.0,2.0,1350000.0,1900,2150.0,628.0,9.0,0.0
3,1.0,1.0,629000.0,1903,500.0,1258.0,9.0,0.0
4,0.0,1.0,439000.0,1930,500.0,878.0,10.0,0.0
...,...,...,...,...,...,...,...,...
487,5.0,2.5,1800000.0,1890,3073.0,586.0,76.0,1.0
488,2.0,1.0,695000.0,1923,1045.0,665.0,106.0,1.0
489,3.0,2.0,1650000.0,1922,1483.0,1113.0,106.0,1.0
490,1.0,1.0,649000.0,1983,850.0,764.0,163.0,1.0


In [ ]:
from math import log1p as log
def is_numeric(value):
    return isinstance(value, (int, float))

class Question:## data type we need for binary classification
  def __init__(self, feature = "", value = 0, label=""):
    self.feature = feature
    self.partition_value = value
    self.label = label

  def entropy(self, df):
    if(len(df) == 0):
      return 0.0
    p_a = float(len(df[df[self.label] == 1])) / float(len(df))
    p_b = float(len(df[df[self.label] == 0])) / float(len(df))
    return -1*(p_a * log(p_a) + p_b * log(p_b))

  def information_gain(self, parent, left_child, right_child):
    return self.entropy(parent) - (len(left_child) / len(parent)) * self.entropy(left_child) - (len(right_child) / len(parent)) * self.entropy(right_child)
  
  def split(self, df):
    parent = df
    left_child = df[df[self.feature] >= self.partition_value] if(is_numeric(self.partition_value)) else df[df[self.feature] == self.partition_value]
    right_child = df[df[self.feature] < self.partition_value] if(is_numeric(self.partition_value)) else df[df[self.feature] !=  self.partition_value]
    ig = self.information_gain(parent, left_child, right_child)
    return (left_child, right_child, ig)
  
  def __repr__(self):
        condition = "=="
        if is_numeric(self.partition_value):
            condition = ">="
        return "Is %5s %s %5s?" % (
            self.feature, condition, str(self.partition_value))

In [ ]:
print(Question('price', 649000, 'Class').split(df))

(     beds  bath      price  ...  price_per_sqft  elevation  Class
0     2.0   1.0   999000.0  ...           999.0       10.0    0.0
1     2.0   2.0  2750000.0  ...          1939.0        0.0    0.0
2     2.0   2.0  1350000.0  ...           628.0        9.0    0.0
7     1.0   1.0   975000.0  ...          1083.0       10.0    0.0
8     1.0   1.0   975000.0  ...          1083.0       12.0    0.0
..    ...   ...        ...  ...             ...        ...    ...
487   5.0   2.5  1800000.0  ...           586.0       76.0    1.0
488   2.0   1.0   695000.0  ...           665.0      106.0    1.0
489   3.0   2.0  1650000.0  ...          1113.0      106.0    1.0
490   1.0   1.0   649000.0  ...           764.0      163.0    1.0
491   3.0   2.0   995000.0  ...           762.0      216.0    1.0

[421 rows x 8 columns],      beds  bath     price  year_built    sqft  price_per_sqft  elevation  Class
3     1.0   1.0  629000.0        1903   500.0          1258.0        9.0    0.0
4     0.0   1.0  43900

In [ ]:
def best_question(df):
  features = df.columns[:-1]
  label = 'Class'
  max_gain = 0.00
  my_question = None
  left_child = None
  right_child = None
  for col in features:
    values = set(df[col])
    for val in values:
      question_i = Question(col, val, label)
      lt, rt, ig = question_i.split(df)
      if(ig > max_gain + 1e-10):
        left_child = lt
        right_child = rt
        my_question = question_i
        max_gain = ig
  return left_child, right_child, my_question

def best_prob(df): # for binary classification
  one, zero = len(df[df['Class'] == 1])/ len(df), len(df[df['Class'] == 0])/ len(df)
  if (one > zero + 1e-50):
      return one, 1
  return zero, 0

In [ ]:

lt, rt, q = best_question(df)
print(q)
rt

Is elevation >=  31.0?


,beds,bath,price,year_built,sqft,price_per_sqft,elevation,Class
0,2.0,1.0,999000.0,1960,1000.0,999.0,10.0,0.0
1,2.0,2.0,2750000.0,2006,1418.0,1939.0,0.0,0.0
2,2.0,2.0,1350000.0,1900,2150.0,628.0,9.0,0.0
3,1.0,1.0,629000.0,1903,500.0,1258.0,9.0,0.0
4,0.0,1.0,439000.0,1930,500.0,878.0,10.0,0.0
...,...,...,...,...,...,...,...,...
479,3.0,2.0,2219000.0,2012,1921.0,1155.0,13.0,1.0
480,1.0,1.0,788000.0,2004,903.0,873.0,4.0,1.0
481,2.0,2.0,1950000.0,1995,1930.0,1010.0,4.0,1.0
482,0.0,1.0,539000.0,2000,709.0,760.0,5.0,1.0


In [ ]:
lt

,beds,bath,price,year_built,sqft,price_per_sqft,elevation,Class
154,2.0,2.0,1350000.0,1931,1300.0,1038.0,36.0,0.0
164,1.0,1.0,650000.0,1907,720.0,903.0,32.0,0.0
180,1.0,1.0,499999.0,2011,669.0,747.0,35.0,0.0
181,1.0,1.0,749000.0,2011,762.0,983.0,35.0,0.0
184,2.0,1.0,559900.0,1925,1200.0,467.0,51.0,0.0
...,...,...,...,...,...,...,...,...
487,5.0,2.5,1800000.0,1890,3073.0,586.0,76.0,1.0
488,2.0,1.0,695000.0,1923,1045.0,665.0,106.0,1.0
489,3.0,2.0,1650000.0,1922,1483.0,1113.0,106.0,1.0
490,1.0,1.0,649000.0,1983,850.0,764.0,163.0,1.0


In [ ]:
## make a tree 
class tree:##Simple implementation with greedy algorithm
  def __init__(self):
    self.ques = None
    self.left = None
    self.right = None
    self.prob = None
    self.label = None

  def build(self, df, threshold = 1):
    if(len(df) == 0):
      return
    lt_df,rt_df,self.ques = best_question(df)
    self.prob, self.label = best_prob(df)
    if(self.ques == None or len(lt_df) < threshold or len(rt_df) < threshold):
      return
    self.left = tree()
    self.right = tree()
    self.left.build(lt_df.loc[:], threshold = threshold)
    self.right.build(rt_df.loc[:], threshold = threshold)

  def query(self, sub_df):
    sub_df['predicted'] = self.label
    if(self.ques == None or len(sub_df) == 0):
      return sub_df
    lt_df, rt_df, _ = self.ques.split(sub_df)
    if(self.left != None):
      lt_df = self.left.query(lt_df.loc[:])
    if(self.right != None):
      rt_df = self.right.query(rt_df.loc[:])
    return pd.concat([lt_df, rt_df])

In [ ]:
df = df.reindex(np.random.permutation(len(df)))
num_train = int(len(df)* 0.8)
train_data = df[0:num_train]
test_data = df[num_train:]
root = tree()
root.build(train_data.loc[:], threshold = 7.)
np_test_data = root.query(test_data.loc[:]).sort_index().values



In [ ]:
np_test_data = np_test_data[:, -2:]
confusion_matrix = np.zeros([2, 2])
for actual, predicted in np_test_data:
  confusion_matrix[int(actual)][int(predicted)] += 1
confusion_matrix

array([[41.,  1.],
       [ 5., 52.]])

In [ ]:
accuracy = (confusion_matrix[0,0] + confusion_matrix[1, 1] ) / (confusion_matrix[0, 0] + confusion_matrix[0, 1] + confusion_matrix[1, 0] + confusion_matrix[1, 1])
recall = (confusion_matrix[1, 1]) / (confusion_matrix[1, 1] + confusion_matrix[1, 0])
precision = (confusion_matrix[1, 1]) / (confusion_matrix[0,1] + confusion_matrix[1,1])
accuracy * 100, recall * 100, precision * 100 

(93.93939393939394, 91.22807017543859, 98.11320754716981)